Данный ноутбук содержит в себе части кода, которые нужны для предобработки данных.

# Библиотеки

In [1]:
import pandas as pd
import cv2
import os
import csv
import imageio
import numpy as np
from medpy.io import load
import shutil
from PIL import Image, ImageDraw, ImageOps

# Предобработки

### 1) Создание нового csv файла, в котором Mass и Nodule объединены в один класс

Файл Data_Entry_2017_v2020.csv вы можете скачать по ссылке: https://nihcc.app.box.com/v/ChestXray-NIHCC

In [2]:
df = pd.read_csv('Data Kaggle/Data_Entry_2017_v2020.csv')

def replace_labels(label):
    if 'Mass' in label and 'Nodule' in label:
        return label.replace('Mass', '').replace('Nodule', 'Nodule/Mass').replace('||', '|').strip('|')
    elif 'Mass' in label:
        return label.replace('Mass', 'Nodule/Mass')
    elif 'Nodule' in label:
        return label.replace('Nodule', 'Nodule/Mass')
    else:
        return label

df['Finding Labels'] = df['Finding Labels'].apply(replace_labels)
df.to_csv('Data Kaggle/Data_Entry_2017_v2020_edit.csv', index=False)

### 2) Расчёт размеров Bbox для патологий Nodule и Mass в сантиметрах

In [3]:
data_entry = pd.read_csv('Data Kaggle/Data_Entry_2017_v2020.csv')
bbox_list = pd.read_csv('Data Kaggle/BBox_List_2017.csv')

filtered_bbox = bbox_list[bbox_list['Finding Label'].isin(['Nodule', 'Mass'])]

merged_data = pd.merge(filtered_bbox, data_entry, on='Image Index')

merged_data['w_cm'] = merged_data['w'] * merged_data['OriginalImagePixelSpacing[x']
merged_data['h_cm'] = merged_data['h]'] * merged_data['y]']

final_data = merged_data[['Image Index', 'Finding Label', 'w_cm', 'h_cm']]

final_data.to_csv('Data Kaggle/BBox_size_cm.csv', index=False)

### 3) Преобразование данных NODE21 из .mha в .png для изображений с Nodule

In [4]:
# Путь к директории с исходными изображениями
source_dir = 'NODE21/dataset_node21/cxr_images/original_data/images'

# Путь к директории для сохранения обработанных изображений
target_dir = 'NODE21/dataset_node21/cxr_images/original_data/png_Nodule'

# Путь к CSV-файлу с соответствием имен
csv_file = 'NODE21/dataset_node21/cxr_images/original_data/filenames_orig_and_new.csv'

# Словарь для сопоставления node21_img_id с original_image_name
id_name_mapping = {}

# Чтение данных из CSV-файла и заполнение словаря
with open(csv_file, mode='r') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        id_name_mapping[row['node21_img_id']] = row['original_image_name']

# Проверка наличия директории для сохранения изображений
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Обход всех файлов в исходной директории
for filename in os.listdir(source_dir):
    if filename.endswith('.mha'):
        # Получение original_image_name из словаря
        original_name = id_name_mapping.get(filename.replace('.mha', ''), None)
        if original_name:
            # Загрузка изображения
            image_data, image_header = load(os.path.join(source_dir, filename))

            # Поворот изображения на 90 градусов вправо
            image_data_rotated = np.rot90(image_data, -1)

            # Нормализация изображения для удаления прозрачности
            image_data_normalized = ((image_data_rotated - np.min(image_data_rotated)) * (255 / (np.max(image_data_rotated) - np.min(image_data_rotated)))).astype(np.uint8)

            # Сохранение обработанного изображения в формате PNG с original_image_name
            imageio.imwrite(os.path.join(target_dir, original_name + '.png'), image_data_normalized)

### 4) Преобразование данных NODE21 из .mha в .png для изображений без Nodule

In [5]:
# Путь к директории с исходными изображениями
source_dir = 'NODE21/dataset_node21/cxr_images/original_data/images'

# Путь к директории для сохранения обработанных изображений
target_dir = 'NODE21/dataset_node21/cxr_images/original_data/png_clean'

# Проверка наличия директории для сохранения изображений
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Обход всех файлов в исходной директории
for filename in os.listdir(source_dir):
    if filename.startswith('c') and filename.endswith('.mha'):
        # Загрузка изображения
        image_data, image_header = load(os.path.join(source_dir, filename))

        # Поворот изображения на 90 градусов вправо
        image_data_rotated = np.rot90(image_data, -1)

        # Нормализация изображения для удаления прозрачности
        image_data_normalized = ((image_data_rotated - np.min(image_data_rotated)) * (255 / (np.max(image_data_rotated) - np.min(image_data_rotated)))).astype(np.uint8)

        # Сохранение обработанного изображения в формате PNG
        imageio.imwrite(os.path.join(target_dir, filename.replace('.mha', '.png')), image_data_normalized)

###  5) Копировние данных XRay Chest Nodule, где возраст от 18 лет

In [6]:
# Загрузка данных из CSV-файла
data = pd.read_csv('Data Kaggle/Data_Entry_2017_v2020_edit.csv')

# Путь к папке с исходными изображениями
source_folder = 'More Bbox Nodule/From kaggle/'

# Путь к папке назначения
destination_folder = 'More Bbox Nodule/vs/'

# Создание папки назначения, если она не существует
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Копирование изображений
for index, row in data.iterrows():
    # Проверка возраста пациента
    if row['Patient Age'] >= 18:
        # Полный путь к исходному изображению
        source_image = os.path.join(source_folder, row['Image Index'])
        # Полный путь к изображению в папке назначения
        destination_image = os.path.join(destination_folder, row['Image Index'])

        # Копирование изображения, если оно существует
        if os.path.isfile(source_image):
            shutil.copy2(source_image, destination_image)
            print(f'Изображение {row["Image Index"]} скопировано.')
        else:
            print(f'Изображение {row["Image Index"]} не найдено в исходной папке.')

### 6) Копирование изображений из XRay Chest Nodule, которых нет в NODE21

In [7]:
# Путь к папке с исходными изображениями
source_folder = 'More Bbox Nodule/vs/'

# Путь к папке, где уже есть некоторые изображения
existing_images_folder = 'NODE21/dataset_node21/cxr_images/original_data/png_Nodule/'

# Путь к папке назначения
destination_folder = 'More Bbox Nodule/merge/'

# Создание папки назначения, если она не существует
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Получение списка имен файлов в папке с существующими изображениями
existing_images = set(os.listdir(existing_images_folder))

# Копирование изображений
for image_name in os.listdir(source_folder):
    # Проверка, существует ли изображение в папке с существующими изображениями
    if image_name not in existing_images:
        # Полный путь к исходному изображению
        source_image = os.path.join(source_folder, image_name)
        # Полный путь к изображению в папке назначения
        destination_image = os.path.join(destination_folder, image_name)

        # Копирование изображения
        shutil.copy2(source_image, destination_image)
        print(f'Изображение {image_name} скопировано.')
    else:
        print(f'Изображение {image_name} уже существует в папке назначения.')

### 7) Предобработка масок для данных Шэньчжэнь и Монтгомери, чтобы сердце также попадало в сегментацию

In [8]:
def process_masks(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    mask_files = os.listdir(input_folder)

    for filename in mask_files:
        mask_path = os.path.join(input_folder, filename)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        original_mask = mask.copy()
        no_white_col = find_no_white_col(mask)
        red_row, right_col_with_white = find_right_col_with_white(mask, no_white_col)
        bottom_row_with_white = find_bottom_row_with_white(mask, right_col_with_white)
        modified_mask = fill_white_area(mask.copy(), right_col_with_white, bottom_row_with_white, red_row)
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, modified_mask)

    print("Обработка завершена.")

def find_no_white_col(mask):
    center = mask.shape[1] // 2
    no_white_col = 0
    for i in range(mask.shape[1]):
        if i % 2 == 0:
            col = center + i // 2
        else:
            col = center - i // 2
        if not np.any(mask[:, col] == 255):
            no_white_col = col
            break
    return no_white_col

def find_right_col_with_white(mask, start_col):
    for col in range(start_col, mask.shape[1]):
        if np.any(mask[:, col] == 255):
            row = np.argmax(mask[:, col] == 255)
            return (row, col)
    return (-1, -1) 

def find_bottom_row_with_white(mask, start_col):
    for row in range(mask.shape[0] - 1, -1, -1):
        if np.any(mask[row, start_col:] == 255):
            return row
    return -1

def fill_white_area(mask, blue_col, green_row, red_row):
    if blue_col == -1 or green_row == -1:
        return mask

    for row in range(green_row, red_row, -1):
        for col in range(blue_col, mask.shape[1]):
            if mask[row, col] == 255:
                break
            mask[row, col] = 255
    return mask

process_masks('Data Segmentation Lung/data/Lung Segmentation/masks', 'Data Segmentation Lung/data/Lung Segmentation/masks_heart')

### 8) Коверитруем из bmp в png и инвертируем изображение для данных JSRT

In [9]:
src_dir = 'JSRT/org'
dst_dir = 'JSRT/org_png'

os.makedirs(dst_dir, exist_ok=True)

for filename in os.listdir(src_dir):
    if filename.endswith('.bmp'):
        img = Image.open(os.path.join(src_dir, filename))
        img = ImageOps.invert(img)
        img.save(os.path.join(dst_dir, filename[:-4] + '.png'))

### 9) Для данных JSRT сделаем маски для сегментации лёгких и сердца

In [10]:
os.makedirs('JSRT/label_cut', exist_ok=True)
files = os.listdir('JSRT/label')

for file in files:
    mask = cv2.imread(f'JSRT/label/{file}', cv2.IMREAD_GRAYSCALE)

    heart_mask = np.copy(mask)
    heart_mask[(heart_mask == 85)] = 255
    heart_mask[heart_mask != 255] = 0

    center = mask.shape[1]//2
    no_white_col = 0

    for i in range(mask.shape[1]):
        if i % 2 == 0:
            col = center + i // 2
        else:
            col = center - i // 2
        if not np.any(mask[:, col] == 255):
            no_white_col = col
            break

    left_most = 0
    right_most = 0

    for i in range(no_white_col, -1, -1):
        if np.any(mask[:, i] == 255):
            left_most = i + 1  
            break

    for i in range(no_white_col, mask.shape[1]):
        if np.any(mask[:, i] == 255):
            right_most = i - 1  
            break

    heart_mask[:, left_most:right_most] = 0

    cv2.imwrite(f'JSRT/label_cut/{file}', heart_mask)

### 10) Создание датасета для данных RSNA

In [11]:
rsna_df = pd.read_csv('Data rsna/lung-segmentation/train.csv')

def add_path(image_name):
    return 'Data rsna/lung-segmentation/images/' + image_name.split('.')[0] + '/image.png'


def add_path_to_label(label_name):
    return 'Data rsna/lung-segmentation/masks/' + label_name

rsna_df['label'] = rsna_df['label'].apply(add_path_to_label)
rsna_df['image'] = rsna_df['image'].apply(add_path)

rsna_df.to_csv('Data rsna/lung-segmentation/updated_train.csv', index=False)

### 11) Сохранение данных из train и test в один файл для XRay Chest Nodule

In [12]:
train = pd.read_csv('More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/train.csv')
test = pd.read_csv('More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/test.csv')

combined = pd.concat([train, test])

combined.to_csv('More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/combined.csv', index=False)

In [13]:
data = pd.read_csv('More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/combined.csv')
data['filename'] = data['filename'].apply(lambda x: "_".join(x.split("_")[:2]) + ".png")
data.to_csv('More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/combined.csv', index=False)

In [14]:
file_path = 'More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/combined.csv'
df = pd.read_csv(file_path)

df['width'] = df['xmax'] - df['xmin']
df['height'] = df['ymax'] - df['ymin']

new_df = pd.DataFrame({
    'height': df['height'],
    'img_name': df['filename'],
    'label': 1,
    'width': df['width'],
    'x': df['xmin'],
    'y': df['ymin']
})

output_file_path = 'More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/transformed_combined.csv'
new_df.to_csv(output_file_path, index=False)

### 12) Сохранение изображений в формате .png для XRay Chest Nodule

train и test это файлы _annotation.csv

In [15]:
src_dir = "More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/train/"
dst_dir = "More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/png/"

if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

for filename in os.listdir(src_dir):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(src_dir, filename))
        base_filename = "_".join(filename.split("_")[:2])
        img.save(os.path.join(dst_dir, base_filename + ".png"))

In [16]:
src_dir = "More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/test/"
dst_dir = "More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/png/"

if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

for filename in os.listdir(src_dir):
    if filename.endswith(".jpg"):
        img = Image.open(os.path.join(src_dir, filename))
        base_filename = "_".join(filename.split("_")[:2])
        img.save(os.path.join(dst_dir, base_filename + ".png"))

### 13) Сохранить бинарные маски Bbox для XRay Chest Nodule

In [17]:
data = pd.read_csv('More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/combined.csv')

mask_dir = 'More Bbox Nodule/CXR.v1-chestx-ray_zhangjin_kaggle.tensorflow/png_mask/'
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)

grouped = data.groupby('filename')

for filename, group in grouped:
    mask = np.zeros((group['height'].iloc[0], group['width'].iloc[0]), dtype=np.uint8)

    for index, row in group.iterrows():
        mask[row['ymin']:row['ymax'], row['xmin']:row['xmax']] = 255

    mask_filename = os.path.join(mask_dir, filename)
    cv2.imwrite(mask_filename, mask)

### 14) Создание объединённого csv для NODE21

In [18]:
metadata_df = pd.read_csv('NODE21/dataset_node21/cxr_images/original_data/metadata.csv')
filenames_df = pd.read_csv('NODE21/dataset_node21/cxr_images/original_data/filenames_orig_and_new.csv')

if 'Unnamed: 0' in metadata_df.columns:
    metadata_df = metadata_df.drop(columns=['Unnamed: 0'])

mapping_dict = dict(zip(filenames_df['node21_img_id'], filenames_df['original_image_name']))

def update_img_name(img_name):
    if img_name.startswith('n') and img_name[:-4] in mapping_dict:
        return mapping_dict[img_name[:-4]] + '.png'
    elif img_name.startswith('c'):
        return img_name[:-4] + '.png'
    else:
        return img_name

metadata_df['img_name'] = metadata_df['img_name'].apply(update_img_name)
metadata_df.to_csv('NODE21/dataset_node21/cxr_images/original_data/updated_metadata.csv', index=False)

### 15) Сохранить бинарные маски Bbox для NODE21

In [19]:
metadata_file = 'NODE21/dataset_node21/cxr_images/original_data/updated_metadata.csv'
output_folder = 'NODE21/dataset_node21/cxr_images/original_data/png_mask/'
os.makedirs(output_folder, exist_ok=True)

df = pd.read_csv(metadata_file)
image_groups = df.groupby('img_name')

for img_name, group in image_groups:
    # Пропустить изображения, имена которых начинаются с 'c'
    if img_name.startswith('c'):
        continue

    image_path = os.path.join('NODE21/dataset_node21/cxr_images/original_data/png_Nodule/', img_name)

    if not os.path.exists(image_path):
        print(f"Файл не найден: {image_path}")
        continue

    image = Image.open(image_path)
    image_width, image_height = image.size

    # Создание пустой маски
    mask = Image.new('L', (image_width, image_height), 0)
    draw = ImageDraw.Draw(mask)

    # Нарисовать прямоугольники для каждого bbox на маске
    for index, row in group.iterrows():
        x, y, width, height = row['x'], row['y'], row['width'], row['height']
        draw.rectangle([x, y, x + width, y + height], fill=255)

    # Отзеркаливание маски по оси y
    mask_np = np.array(mask)
    mirrored_mask_np = np.flip(mask_np, axis=1)
    mirrored_mask = Image.fromarray(mirrored_mask_np)

    # Сохранение отзеркаленной маски
    mask_filename = os.path.join(output_folder, os.path.splitext(img_name)[0] + '.png')
    mirrored_mask.save(mask_filename)